In [2]:
from os import listdir
from os.path import isfile, join

mypath = '../Lab1/ustawy/'
ustawy = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
ustawy

['1993_599.txt',
 '1993_602.txt',
 '1993_645.txt',
 '1993_646.txt',
 '1994_150.txt',
 '1994_195.txt',
 '1994_201.txt',
 '1994_214.txt',
 '1994_215.txt',
 '1994_288.txt',
 '1994_294.txt',
 '1994_331.txt',
 '1994_332.txt',
 '1994_344.txt',
 '1994_362.txt',
 '1994_363.txt',
 '1994_368.txt',
 '1994_369.txt',
 '1994_37.txt',
 '1994_384.txt',
 '1994_385.txt',
 '1994_395.txt',
 '1994_396.txt',
 '1994_419.txt',
 '1994_472.txt',
 '1994_473.txt',
 '1994_516.txt',
 '1994_536.txt',
 '1994_591.txt',
 '1994_592.txt',
 '1994_600.txt',
 '1994_601.txt',
 '1994_602.txt',
 '1994_615.txt',
 '1994_627.txt',
 '1994_645.txt',
 '1994_669.txt',
 '1994_670.txt',
 '1994_704.txt',
 '1994_97.txt',
 '1995_1.txt',
 '1995_121.txt',
 '1995_141.txt',
 '1995_143.txt',
 '1995_152.txt',
 '1995_164.txt',
 '1995_184.txt',
 '1995_2.txt',
 '1995_208.txt',
 '1995_221.txt',
 '1995_242.txt',
 '1995_261.txt',
 '1995_29.txt',
 '1995_31.txt',
 '1995_310.txt',
 '1995_334.txt',
 '1995_335.txt',
 '1995_368.txt',
 '1995_390.txt',
 '199

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch(['http://localhost:9200/'], verify_certs=True, timeout=30)

In [5]:
es.ping()

True

# Zad 3

In [6]:
analyzer = {
    "my_analyzer": {
        "tokenizer": "standard",
        "filter": [ "morfologik_stem", "synonym_filter", "lowercase" ]
    }
}
filters = {
    "synonym_filter": {
        "type": "synonym",
        "lenient": True,
        "synonyms": [ 
                    "kpk, kodeks postępowania karnego",
                    "kpc, kodeks postępowania cywilnego",
                    "kk, kodeks karny",
                    "kc, kodeks cywilny"]
    }
}

In [7]:
settings_body = {
    "settings": {
        "index": {
            "analysis": {
                "analyzer": analyzer,
                "filter": filters
            }
        }
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            "text": {
                "type": "text",
                "term_vector": "with_positions_offsets",
                "analyzer": "my_analyzer"
            }
        }
    }
}

# Zad 4

In [8]:
my_index = 'my_index'
es.indices.delete(index=my_index, ignore=[400, 404])
es.indices.create(
    index=my_index,
    body=settings_body
)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'}

# Zad 5

In [9]:
import json

documents_ids = []
for ustawa in ustawy:
    with open(join(mypath, ustawa), encoding='utf-8') as f:
        text = f.read()
        record = json.dumps({
            "text": text
        })
        outcome = es.index(index=my_index, body=record)
        documents_ids.append(outcome)

# Zad 6

In [10]:
res_count = es.count(index=my_index, body={
    "query": {
        "match": {
            "text": {
                "query": "ustawa"
            }
        }
    }
})

In [11]:
print("Liczba dokumentów ze słowem \"ustawa\" w dowolnej formie: ", res_count['count'])

Liczba dokumentów ze słowem "ustawa" w dowolnej formie:  1178


# Zad 7

In [12]:
def count_word_in_docs(word):
    res = es.search(index=my_index, body={
        "size": 10000,
        "query": {
            "query_string": {
                "fields": ["text"],
                "query": word,
                "analyzer": "my_analyzer"
            }
        }
    })

    ids = [h['_id'] for h in res['hits']['hits']]
    sum = 0

    for id in ids:
        vectors = es.termvectors(
                index=my_index,
                id=id,
                fields=['text'], 
                field_statistics=False
        )
        sum += vectors['term_vectors']['text']['terms'][word]['term_freq']
    return sum

In [13]:
ustawa_count = count_word_in_docs('ustawa')

In [14]:
print("Liczba wystąpień słowa \"ustawa\" w każdej formie: ", ustawa_count)

Liczba wystąpień słowa "ustawa" w każdej formie:  24934


# Zad 8

In [15]:
ustaw_count = count_word_in_docs('ustawić')

In [16]:
print("Liczba wystąpień słowa \"ustaw\" w każdej formie: ", ustaw_count)

Liczba wystąpień słowa "ustaw" w każdej formie:  913


# Zad 9

In [58]:
res_count = es.count(index=my_index, body={
    "query": {
        "match_phrase": {
            "text": {
                "query": "kodeks postępowania cywilnego"
            }
        }
    }
})

In [59]:
print("Liczba dokumentów z frazą \"kodeks postępowania cywilnego\" w dowolnej formie: ", res_count['count'])

Liczba dokumentów z frazą "kodeks postępowania cywilnego" w dowolnej formie:  99


# Zad 10

In [56]:
res_count = es.count(index=my_index, body={
    "query": {
        "match_phrase": {
            "text": {
                "query": "wchodzi w życie",
                "slop": 2
            }
        }
    }
})

In [57]:
print("Liczba dokumentów z frazą \"wchodzi w życie\" w dowolnej formie: ", res_count['count'])

Liczba dokumentów z frazą "wchodzi w życie" w dowolnej formie:  1174


# Zad 11

In [44]:
res = es.search(index=my_index, body={
    "size": 10,
    "query": {
        "match": {
            "text": {
                "query": "konstytucja"
            }
        }
    },
    "highlight": {
        "fields": {
            "text": {}
        },
        "boundary_scanner": "sentence",
        "fragment_size": 40,
        "number_of_fragments": 3,
        "order": "score",
        "type": "unified"
    }
})

hits = res['hits']['hits']

In [45]:
id_to_name_dic = {}
for i in range(len(ustawy)):
     id_to_name_dic[documents_ids[i]['_id']] = ustawy[i]

In [46]:
for hit in hits:
    print(f"File: {id_to_name_dic[hit['_id']]}, score: {hit['_score']}")

File: 1997_629.txt, score: 6.869192
File: 2000_443.txt, score: 6.663512
File: 1997_604.txt, score: 6.6323376
File: 1996_350.txt, score: 6.627432
File: 1997_642.txt, score: 6.2523136
File: 2001_23.txt, score: 6.056943
File: 1996_199.txt, score: 5.9268084
File: 1999_688.txt, score: 5.848971
File: 1997_681.txt, score: 5.4654307
File: 2001_1082.txt, score: 5.4654307


# Zad 12

In [54]:
for hit in hits:
    highlights = hit['highlight']['text']
    for highlight in highlights:
        print(highlight)
        print("--------------------------------------------------------------------------------------------------------------")

Do zgłoszenia projektu <em>Konstytucji</em> załącza
--------------------------------------------------------------------------------------------------------------
Projekty <em>Konstytucji</em> wniesione na podstawie
--------------------------------------------------------------------------------------------------------------
Zasady, na których opierać się ma <em>Konstytucja</em>
--------------------------------------------------------------------------------------------------------------
określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
--------------------------------------------------------------------------------------------------------------
określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
--------------------------------------------------------------------------------------------------------------
której mowa w art. 89
  ust. 1 i art. 90 <em>Konstytucji</em>
---------------------------------------------------------------------------------------------------